In [1]:
from mlrun.run import new_function,code_to_function
from mlrun import get_or_create_ctx
from mlrun import mount_v3io,NewTask,mlconf

In [2]:
docker_registry = 'docker-registry.default-tenant.app.mdl1231.iguazio-cd2.com:80'

In [3]:
#fn = new_function(command='spark://marcelo/sparkk8s#local:///User/spark-jdbc.py')
pyscript = "/User/mlrun_demos/spark/train.py"

In [4]:
fn = new_function(kind='spark',name='iris-trainer',command='urihoenig/spark-app:2.4.4-2.9.0-0.0.2#local://'+ pyscript)
fn.with_limits(mem="512m")
fn.with_requests(cpu=1)

In [5]:
fn.spec.rundb = 'http://mlrun-api:8080'
mlconf.dbpath = 'http://mlrun-api:8080'
fn.with_igz_spark('2.8_b3506_20191217042239')
fn.spec.env += [{'name': 'MLRUN_DBPATH','value':'http://mlrun-api:8080'}]

In [6]:
fn.apply(mount_v3io())

In [7]:
print(fn.to_yaml())


kind: spark
metadata:
  name: iris-trainer
  project: default
spec:
  command: local:///User/mlrun_demos/spark/train.py
  args: []
  image: urihoenig/spark-app:2.4.4-2.9.0-0.0.2
  volumes:
  - flexVolume:
      driver: v3io/fuse
      options:
        accessKey: ed6d0bd0-7bd9-4eb2-b122-676dff4fe912
        container: ''
        subPath: ''
    name: v3io-fuse
  - hostPath:
      path: /dev/shm/default-tenant
      type: ''
    name: shm
  - hostPath:
      path: /var/run/iguazio/dayman/default-tenant
      type: ''
    name: v3iod-comm
  - emptyDir: {}
    name: daemon-health
  - configMap:
      defaultMode: 420
      name: spark-operator-v3io-config
    name: v3io-config
  - flexVolume:
      driver: v3io/fuse
      options:
        accessKey: ed6d0bd0-7bd9-4eb2-b122-676dff4fe912
        container: users
        subPath: /admin
    name: v3io
  volume_mounts:
  - mountPath: /v3io
    name: v3io-fuse
  - mountPath: /dev/shm
    name: shm
  - mountPath: /var/run/iguazio/dayman
    name

In [8]:
#task = NewTask(name='iris-trainer',out_path='/User/mlrun-data/data').with_params(train_data="file:///User/mlrun_demos/spark/data/iris.csv").with_hyper_params({"regularization_rate":[0.02,0.01,0.03],},'max.accuracy')
task = NewTask(name='iris-trainer',
               out_path='/User/mlrun-data/data').with_params(train_data="file:///User/mlrun_demos/spark/data/iris.csv",
                                                             regularization_rate=0.1)


In [9]:
print(task.to_yaml())

spec:
  parameters:
    train_data: file:///User/mlrun_demos/spark/data/iris.csv
    regularization_rate: 0.1
  outputs: []
  output_path: /User/mlrun-data/data
  secret_sources: []
metadata:
  name: iris-trainer
  iteration: 0



In [10]:
fn.run(task,watch=False)

[mlrun] 2020-01-02 19:55:29,875 starting run iris-trainer uid=dc0e223a962a410ba8d5659e3d1faa9f  -> http://mlrun-api:8080


uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...1faa9f,0,Jan 02 19:55:29,completed,iris-trainer,kind=sparkowner=admin,,regularization_rate=0.1train_data=file:///User/mlrun_demos/spark/data/iris.csv,,


to track results use .show() or .logs() or in CLI: 
!mlrun get run dc0e223a962a410ba8d5659e3d1faa9f  , !mlrun logs dc0e223a962a410ba8d5659e3d1faa9f 
[mlrun] 2020-01-02 19:56:27,478 run executed, status=completed


In [ ]:
!mlrun get run 3125cee1ec1546d1b15b4a415180c2ba